In [2]:
import sklearn
import pandas as pd
import numpy as np
from __future__ import division
import collections
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import tree
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from keras.utils import np_utils
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import sys
from sklearn.ensemble import GradientBoostingRegressor
import math
import csv
%matplotlib inline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
import urllib
from sklearn.svm import LinearSVC
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Load All the Data.

In [3]:
reg_season_compact = pd.read_csv('./DataFiles/RegularSeasonCompactResults.csv')
reg_season_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [4]:
reg_season_detailed = pd.read_csv('./DataFiles/RegularSeasonDetailedResults.csv')
reg_season_detailed.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [5]:
seasons = pd.read_csv('./DataFiles/Seasons.csv')
seasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [6]:
teams = pd.read_csv('./DataFiles/Teams.csv')
teamList = teams['TeamName'].tolist() #will use later.
teams.tail()

,TeamID,TeamName,FirstD1Season,LastD1Season
359,1460,Wright St,1988,2018
360,1461,Wyoming,1985,2018
361,1462,Xavier,1985,2018
362,1463,Yale,1985,2018
363,1464,Youngstown St,1985,2018


In [7]:
#doing a spot check, Scgoooo Ducks 

print(teams[teams['TeamName'] == 'Oregon'])

     TeamID TeamName  FirstD1Season  LastD1Season
231    1332   Oregon           1985          2018


In [8]:
tourney_compact = pd.read_csv('./DataFiles/NCAATourneyCompactResults.csv')
tourney_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [9]:
tourney_detailed = pd.read_csv('./DataFiles/NCAATourneyDetailedResults.csv')
tourney_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [10]:
seeds = pd.read_csv('./DataFiles/NCAATourneySeeds.csv')
seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [11]:
#Lets see how Oregon has done throughout the years. 
scgooo_ducks = teams[teams['TeamName'] == 'Oregon'].values[0][0]
seeds[seeds['TeamID'] == scgooo_ducks]

,Season,Seed,TeamID
661,1995,X06,1332
966,2000,W07,1332
1107,2002,X02,1332
1194,2003,Y08,1332
1448,2007,Y03,1332
1520,2008,Y09,1332
1855,2013,Y12,1332
1936,2014,Z07,1332
2004,2015,Z08,1332
2065,2016,Z01,1332


In [12]:
#Now lets check Duke. Dayyyyyyyum.

duke = teams[teams["TeamName"] == 'Duke'].values[0][0]
seeds[seeds['TeamID'] == duke]

,Season,Seed,TeamID
34,1985,Y03,1181
64,1986,W01,1181
164,1987,Y05,1181
193,1988,W02,1181
257,1989,W02,1181
322,1990,W03,1181
417,1991,Y02,1181
448,1992,W01,1181
530,1993,X03,1181
593,1994,X02,1181


In [13]:
seed_slots = pd.read_csv('./DataFiles/NCAATourneySlots.csv')
seed_slots.head() #okay

,Season,Slot,StrongSeed,WeakSeed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [14]:
conferences = pd.read_csv('./DataFiles/ConferenceTourneyGames.csv')
conferences.head()

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID
0,2001,a_sun,121,1194,1144
1,2001,a_sun,121,1416,1240
2,2001,a_sun,122,1209,1194
3,2001,a_sun,122,1359,1239
4,2001,a_sun,122,1391,1273


# Feature Selection and Engineering.

In [15]:
ACC_Teams = ['North Carolina','Virginia','Florida St','Louisville','Notre Dame','Syracuse','Duke','Virginia Tech','Georgia Tech','Miami','Wake Forest','Clemson','NC State','Boston College','Pittsburgh']
Pac12_Teams = ['Arizona','Oregon','UCLA','California','USC','Utah','Washington St','Stanford','Arizona St','Colorado','Washington','Oregon St']
SEC_Teams = ['Kentucky','South Carolina','Florida','Arkansas','Alabama','Tennessee','Mississippi St','Georgia','Ole Miss','Vanderbilt','Auburn','Texas A&M','LSU','Missouri']
Big10_Teams = ['Maryland','Wisconsin','Purdue','Northwestern','Michigan St','Indiana','Iowa','Michigan','Penn St','Nebraska','Minnesota','Illinois','Ohio St','Rutgers']
Big12_Teams = ['Kansas','Baylor','West Virginia','Iowa St','TCU','Kansas St','Texas Tech','Oklahoma St','Texas','Oklahoma']
BigEast_Teams = ['Butler','Creighton','DePaul','Georgetown','Marquette','Providence','Seton Hall','St John\'s','Villanova','Xavier']

In [16]:
def check_P6(team_id):
    team_name = teams.values[team_id-1101][1]
    if (team_name in ACC_Teams or team_name in Pac12_Teams or team_name in SEC_Teams \
    or team_name in Big10_Teams or team_name in Big12_Teams or team_name in BigEast_Teams):
        return 1
    else:
        return 0

In [17]:
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [18]:
def get_team_ID(name):
    return teams[teams['TeamName'] == name].values[0][0]

In [19]:
get_team_ID('Akron') #spot check

1103

In [20]:
def get_team_name(team_id):
    return teams[teams['TeamID'] == team_id].values[0][1]

In [21]:
get_team_name(1101)

'Abilene Chr'

In [22]:
def get_list_for_URL(team_list):
    team_list = [x.lower() for x in team_list]
    team_list = [t.replace(' ', '-') for t in team_list]
    team_list = [t.replace('st', 'state') for t in team_list]
    team_list = [t.replace('northern-dakota', 'north-dakota') for t in team_list]
    team_list = [t.replace('nc-', 'north-carolina-') for t in team_list]
    team_list = [t.replace('fl-', 'florida-') for t in team_list]
    team_list = [t.replace('ga-', 'georgia-') for t in team_list]
    team_list = [t.replace('lsu', 'louisiana-state') for t in team_list]
    team_list = [t.replace('maristate', 'marist') for t in team_list]
    team_list = [t.replace('stateate', 'state') for t in team_list]
    team_list = [t.replace('northernorthern', 'northern') for t in team_list]
    team_list = [t.replace('usc', 'southern-california') for t in team_list]
    base = 'http://www.sports-reference.com/cbb/schools/'
    for team in team_list:
        url = base + team + '/'
get_list_for_URL(teamList)

In [23]:
# Function for handling the annoying cases of Florida and FL, as well as State and St
def handle_cases(arr):
    indices = []
    list_len = len(arr)
    for i in range(list_len):
        if (arr[i] == 'St' or arr[i] == 'FL'):
            indices.append(i)
    for p in indices:
        arr[p-1] = arr[p-1] + ' ' + arr[p]
    for i in range(len(indices)): 
        arr.remove(arr[indices[i] - i])
    return arr

# Get Season Vectors


In [24]:
reg_season_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [25]:
seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [26]:
reg_season_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [27]:
reg_season_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [28]:
def handle_different_csv(df):
    # The stats CSV is a lit different in terms of naming so below is just some data cleaning
    df['School'] = df['School'].replace('(State)', 'St', regex=True) 
    df['School'] = df['School'].replace('Albany (NY)', 'Albany NY') 
    df['School'] = df['School'].replace('Boston University', 'Boston Univ')
    df['School'] = df['School'].replace('Central Michigan', 'C Michigan')
    df['School'] = df['School'].replace('(Eastern)', 'E', regex=True)
    df['School'] = df['School'].replace('Louisiana St', 'LSU')
    df['School'] = df['School'].replace('North Carolina St', 'NC State')
    df['School'] = df['School'].replace('Southern California', 'USC')
    df['School'] = df['School'].replace('University of California', 'California', regex=True) 
    df['School'] = df['School'].replace('American', 'American Univ')
    df['School'] = df['School'].replace('Arkansas-Little Rock', 'Ark Little Rock')
    df['School'] = df['School'].replace('Arkansas-Pine Bluff', 'Ark Pine Bluff')
    df['School'] = df['School'].replace('Bowling Green St', 'Bowling Green')
    df['School'] = df['School'].replace('Brigham Young', 'BYU')
    df['School'] = df['School'].replace('Cal Poly', 'Cal Poly SLO')
    df['School'] = df['School'].replace('Centenary (LA)', 'Centenary')
    df['School'] = df['School'].replace('Central Connecticut St', 'Central Conn')
    df['School'] = df['School'].replace('Charleston Southern', 'Charleston So')
    df['School'] = df['School'].replace('Coastal Carolina', 'Coastal Car')
    df['School'] = df['School'].replace('College of Charleston', 'Col Charleston')
    df['School'] = df['School'].replace('Cal St Fullerton', 'CS Fullerton')
    df['School'] = df['School'].replace('Cal St Sacramento', 'CS Sacramento')
    df['School'] = df['School'].replace('Cal St Bakersfield', 'CS Bakersfield')
    df['School'] = df['School'].replace('Cal St Northridge', 'CS Northridge')
    df['School'] = df['School'].replace('East Tennessee St', 'ETSU')
    df['School'] = df['School'].replace('Detroit Mercy', 'Detroit')
    df['School'] = df['School'].replace('Fairleigh Dickinson', 'F Dickinson')
    df['School'] = df['School'].replace('Florida Atlantic', 'FL Atlantic')
    df['School'] = df['School'].replace('Florida Gulf Coast', 'FL Gulf Coast')
    df['School'] = df['School'].replace('Florida International', 'Florida Intl')
    df['School'] = df['School'].replace('George Washington', 'G Washington')
    df['School'] = df['School'].replace('Georgia Southern', 'Ga Southern')
    df['School'] = df['School'].replace('Gardner-Webb', 'Gardner Webb')
    df['School'] = df['School'].replace('Illinois-Chicago', 'IL Chicago')
    df['School'] = df['School'].replace('Kent St', 'Kent')
    df['School'] = df['School'].replace('Long Island University', 'Long Island')
    df['School'] = df['School'].replace('Loyola Marymount', 'Loy Marymount')
    df['School'] = df['School'].replace('Loyola (MD)', 'Loyola MD')
    df['School'] = df['School'].replace('Loyola (IL)', 'Loyola-Chicago')
    df['School'] = df['School'].replace('Massachusetts', 'MA Lowell')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    df['School'] = df['School'].replace('Miami (FL)', 'Miami FL')
    df['School'] = df['School'].replace('Miami (OH)', 'Miami OH')
    df['School'] = df['School'].replace('Missouri-Kansas City', 'Missouri KC')
    df['School'] = df['School'].replace('Monmouth', 'Monmouth NJ')
    df['School'] = df['School'].replace('Mississippi Valley St', 'MS Valley St')
    df['School'] = df['School'].replace('Montana St', 'MTSU')
    df['School'] = df['School'].replace('Northern Colorado', 'N Colorado')
    df['School'] = df['School'].replace('North Dakota St', 'N Dakota St')
    df['School'] = df['School'].replace('Northern Illinois', 'N Illinois')
    df['School'] = df['School'].replace('Northern Kentucky', 'N Kentucky')
    df['School'] = df['School'].replace('North Carolina A&T', 'NC A&T')
    df['School'] = df['School'].replace('North Carolina Central', 'NC Central')
    df['School'] = df['School'].replace('Pennsylvania', 'Penn')
    df['School'] = df['School'].replace('South Carolina St', 'S Carolina St')
    df['School'] = df['School'].replace('Southern Illinois', 'S Illinois')
    df['School'] = df['School'].replace('UC-Santa Barbara', 'Santa Barbara')
    df['School'] = df['School'].replace('Southeastern Louisiana', 'SE Louisiana')
    df['School'] = df['School'].replace('Southeast Missouri St', 'SE Missouri St')
    df['School'] = df['School'].replace('Stephen F. Austin', 'SF Austin')
    df['School'] = df['School'].replace('Southern Methodist', 'SMU')
    df['School'] = df['School'].replace('Southern Mississippi', 'Southern Miss')
    df['School'] = df['School'].replace('Southern', 'Southern Univ')
    df['School'] = df['School'].replace('St. Bonaventure', 'St Bonaventure')
    df['School'] = df['School'].replace('St. Francis (NY)', 'St Francis NY')
    df['School'] = df['School'].replace('Saint Francis (PA)', 'St Francis PA')
    df['School'] = df['School'].replace('St. John\'s (NY)', 'St John\'s')
    df['School'] = df['School'].replace('Saint Joseph\'s', 'St Joseph\'s PA')
    df['School'] = df['School'].replace('Saint Louis', 'St Louis')
    df['School'] = df['School'].replace('Saint Mary\'s (CA)', 'St Mary\'s CA')
    df['School'] = df['School'].replace('Mount Saint Mary\'s', 'Mt St Mary\'s')
    df['School'] = df['School'].replace('Saint Peter\'s', 'St Peter\'s')
    df['School'] = df['School'].replace('Texas A&M-Corpus Christian', 'TAM C. Christian')
    df['School'] = df['School'].replace('Texas Christian', 'TCU')
    df['School'] = df['School'].replace('Tennessee-Martin', 'TN Martin')
    df['School'] = df['School'].replace('Texas-Rio Grande Valley', 'UTRGV')
    df['School'] = df['School'].replace('Texas Southern', 'TX Southern')
    df['School'] = df['School'].replace('Alabama-Birmingham', 'UAB')
    df['School'] = df['School'].replace('UC-Davis', 'UC Davis')
    df['School'] = df['School'].replace('UC-Irvine', 'UC Irvine')
    df['School'] = df['School'].replace('UC-Riverside', 'UC Riverside')
    df['School'] = df['School'].replace('Central Florida', 'UCF')
    df['School'] = df['School'].replace('Louisiana-Lafayette', 'ULL')
    df['School'] = df['School'].replace('Louisiana-Monroe', 'ULM')
    df['School'] = df['School'].replace('Maryland-Baltimore County', 'UMBC')
    df['School'] = df['School'].replace('North Carolina-Asheville', 'UNC Asheville')
    df['School'] = df['School'].replace('North Carolina-Greensboro', 'UNC Greensboro')
    df['School'] = df['School'].replace('North Carolina-Wilmington', 'UNC Wilmington')
    df['School'] = df['School'].replace('Nevada-Las Vegas', 'UNLV')
    df['School'] = df['School'].replace('Texas-Arlington', 'UT Arlington')
    df['School'] = df['School'].replace('Texas-San Antonio', 'UT San Antonio')
    df['School'] = df['School'].replace('Texas-El Paso', 'UTEP')
    df['School'] = df['School'].replace('Virginia Commonwealth', 'VA Commonwealth')
    df['School'] = df['School'].replace('Western Carolina', 'W Carolina')
    df['School'] = df['School'].replace('Western Illinois', 'W Illinois')
    df['School'] = df['School'].replace('Western Kentucky', 'WKU')
    df['School'] = df['School'].replace('Western Michigan', 'W Michigan')
    df['School'] = df['School'].replace('Abilene Christian', 'Abilene Chr')
    df['School'] = df['School'].replace('Montana State', 'Montana St')
    df['School'] = df['School'].replace('Central Arkansas', 'Cent Arkansas')
    df['School'] = df['School'].replace('Houston Baptist', 'Houston Bap')
    df['School'] = df['School'].replace('South Dakota St', 'S Dakota St')
    df['School'] = df['School'].replace('Maryland-Eastern Shore', 'MD E Shore')
    return df

In [29]:
def get_season_data(team_id, year):
    #this df will hold the Season info.
    year_df = reg_season_compact[reg_season_compact['Season'] == year]
    #sum points per game.
    games_won = year_df[year_df.WTeamID == team_id]
    total_points_scored = games_won['WScore'].sum()
    games_lost = year_df[year_df.LTeamID == team_id]
    total_games = games_won.append(games_lost)
    num_games = len(total_games.index)
    total_points_scored += games_lost['LScore'].sum()

    
    #Number of points allowed per game.
    total_points_allowed = games_won["LScore"].sum()
    total_points_allowed += games_lost['WScore'].sum()



    stats_SOS = pd.read_csv('./MM_Stats/MM_Stats_'+str(year)+'.csv')
    stats_SOS = handle_different_csv(stats_SOS)
    rankings_df = pd.read_csv('./Rankings/Rankings_'+str(year)+'.csv')
    rankings_df = handle_different_csv(rankings_df)
    
    name = get_team_name(team_id)
    team = stats_SOS[stats_SOS['School']==name]
    team_ranking = rankings_df[rankings_df['School'] == name]
    if (len(team.index) == 0 or len(team_ranking.index) == 0):
        total_3s_made = 0
        total_turnovers = 0
        total_assists = 0
        sos = 0
        total_rebounds = 0
        srs = 0
        total_steals = 0
    else:
        total_3s_made = team['X3P'].values[0]
        total_turnovers = team['TOV'].values[0]
        if (math.isnan(total_turnovers)):
            total_turnovers = 0
        total_assists = team['AST'].values[0]
        if (math.isnan(total_assists)):
            total_assists = 0
        sos = team['SOS'].values[0]
        srs = team['SRS'].values[0]
        total_rebounds = team['TRB'].values[0]
        if (math.isnan(total_rebounds)):
            total_rebounds = 0
        total_steals = team['STL'].values[0]
        if (math.isnan(total_steals)):
            total_steals = 0

    
    #Tourney Seed for a given year.
    tourney_year = seeds[seeds['Season'] == year]
    team_seed = tourney_year[tourney_year['TeamID'] == team_id]
    if (len(team_seed.index) != 0):
        team_seed = team_seed.values[0][1]
        tournament_seed = int(team_seed[1:3])
    else:
        tournament_seed = 25
        
    #finding number of wins
    num_wins = len(games_won.index)
    if num_games == 0:
        avg_points_scored = 0
        avg_points_allowed = 0
        avg_3s_made = 0
        avg_turnovers = 0
        avg_assists = 0
        avg_rebounds = 0
        avg_steals = 0
    else:
        avg_points_scored = total_points_scored / num_games
        avg_points_allowed = total_points_allowed / num_games
        avg_3s_made = total_3s_made / num_games
        avg_turnovers = total_turnovers / num_games
        avg_assists = total_assists / num_games
        avg_rebounds = total_rebounds / num_games
        avg_steals = total_steals / num_games
        
        
    return [num_wins, avg_points_scored, avg_points_allowed, check_P6(team_id),\
           avg_3s_made, avg_assists, avg_turnovers, tournament_seed, avg_rebounds,\
           avg_steals, sos, srs]
    

In [30]:
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [31]:
#spot check

kentucky_id = teams[teams['TeamName'] == 'Kentucky'].values[0][0]
get_season_data(kentucky_id, 2016)

[26,
 79.67647058823529,
 68.26470588235294,
 1,
 7.176470588235294,
 15.058823529411764,
 11.823529411764707,
 4,
 41.029411764705884,
 6.0588235294117645,
 8.84,
 20.23]

In [32]:
def compare(id_1, id_2, year):
    team_1 = get_season_data(id_1, year)
    team_2 = get_season_data(id_2, year)
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

In [33]:
# Spot check, first create a new ID
kansas_id = teams[teams['TeamName'] == 'Kansas'].values[0][0]
compare(kansas_id, kentucky_id, 2016)


[3,
 0.6265597147950075,
 -0.6586452762923329,
 0,
 2.0356506238859184,
 3.24420677361854,
 2.570409982174688,
 -3,
 2.6978609625668426,
 1.6078431372549025,
 2.380000000000001,
 3.6400000000000006]

In [34]:
def make_season_dict(year):
    season_dict = collections.defaultdict(list)
    for team in teamList:
        team_id = teams[teams['TeamName'] == team].values[0][0]
        team_vector = get_season_data(team_id, year)
        season_dict[team_id] = team_vector
        
    return season_dict

In [35]:
def home_advantage(row):
    if (row == 'H'):
        home = 1
    if (row == 'A'):
        home = (-1)
    if (row == 'N'):
        home = 0 
    return home

In [36]:
reg_season_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [37]:
def train_test(years):
    total_num_games = 0
    for year in years:
        season = reg_season_compact[reg_season_compact['Season'] == year]
        total_num_games += len(season.index)
        tourney = tourney_compact[tourney_compact['Season'] == year]
        total_num_games += len(tourney.index)
    num_features = len(get_season_data(1181, 2012)) ##spot check
    
    X_model = np.zeros((total_num_games, num_features +1))
    y_model = np.zeros((total_num_games))
    
    index_counter = 0 
    for year in years:
        team_vectors = make_season_dict(year)
        season = reg_season_compact[reg_season_compact['Season'] == year]
        season_games = len(season.index)
        tourney = tourney_compact[tourney_compact['Season'] == year]
        season_games += len(tourney.index)
        
        X_model_season = np.zeros((season_games, num_features + 1))
        y_model_season = np.zeros((season_games))
        
        counter = 0
        
        for index, row in season.iterrows():
            winning_team = row['WTeamID']
            winning_vector = team_vectors[winning_team]
            losing_team = row['LTeamID']
            losing_vector = team_vectors[losing_team]
            diff = [a - b for a, b in zip(winning_vector, losing_vector)]
            home = home_advantage(row['WLoc'])
            if (counter % 2 == 0):
                diff.append(home)
                X_model_season[counter] = diff
                y_model_season[counter] = 1
            else:
                diff.append(-home)
                X_model_season[counter] = [-p for p in diff]
                y_model_season[counter] = 0
            counter += 1
        
        for index, row in tourney.iterrows():
            winning_team = row["WTeamID"]
            winning_vector = team_vectors[winning_team]
            losing_team = row['LTeamID']
            losing_vector = team_vectors[losing_team]
            diff = [a - b for a, b in zip(winning_vector, losing_vector)]
            home = 0 # all NCAA games are neutral.
            if (counter % 2 == 0):
                diff.append(home)
                X_model_season[counter] = diff
                y_model_season[counter] = 1
            else:
                diff.append(-home)
                X_model_season[counter] = [-p for p in diff]
                y_model_season[counter] = 0
            counter += 1
        
        X_model[index_counter:season_games + index_counter] = X_model_season
        y_model[index_counter:season_games + index_counter] = y_model_season
        index_counter += season_games
        
    return X_model, y_model

In [38]:
def normalizer(arr):
    for i in range(arr.shape[1]):
        min_val = min(arr[:,i])
        max_val = max(arr[:,i])
        arr[:,i] = (arr[:,i] - min_val) / (max_val - min_val)
    return arr    
    

In [39]:
years = range(1993, 2017)
X_model, y_model = train_test(years)
np.save('X_model', X_model)
np.save('y_model', y_model)

In [40]:
import numpy as np
X_model = np.load('./X_Train.npy')
y_model = np.load('./y_train.npy')

In [41]:
X_model.shape

(115113, 13)

In [42]:
y_model.shape

(115113,)

# Different Models to Try.

In [48]:
#model = Sequential()

#model = tree.DecisionTreeClassifier()
#model = tree.DecisionTreeRegressor()
#model = linear_model.LogisticRegression()
#model = linear_model.BayesianRidge()
#model = linear_model.Lasso()
#model = svm.SVC()
#model = svm.SVR()
#model = linear_model.Ridge(alpha = 0.5)
model = AdaBoostClassifier(n_estimators=100)
#model = GradientBoostingClassifier(n_estimators=100)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)
#model = RandomForestClassifier(n_estimators=64)
#model = KNeighborsClassifier(n_neighbors=39)
#neuralNetwork(10)
#model = VotingClassifier(estimators=[('GBR', model1), ('BR', model2), ('KNN', model3)], voting='soft')
#model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)

In [49]:
# model = Sequential()
# model.add(Dense(13, input_dim=X_model.shape[1], activation='relu'))
# model.add(Dense(4, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
def dependency(predictions, test, stat, identifiers):
    difference = test[:, categories.index(stat)]
    plt.scatter(difference, predictions)
    plt.ylabel('Prob. that Team1 Wins')
    plt.xlabel(stat + 'Difference (Team1 - Team2)')
    plt.show()

In [51]:
def feature_importance(identifiers):
    fi = pd.Series(model.feature_importances_, index = my_categories)
    fi /= fi.max()
    fi.sort()
    fi.plot(kind='barh')

    

In [52]:
my_categories = ['Wins', 'PPG', 'PPGA', 'PowerConf', '3PG', 'APG', 'TOP', 'Conference Champ', \
             'Tourney Conference Champ', 'Seed', 'SOS', 'SRS', 'RPG', 'SPG', 'Tourney Appearances',\
                'National Championships', 'Location']

accuracy = []

for i in range(1):
    X_train, X_test, y_train, y_test = train_test_split(X_model, y_model)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    accuracy.append(np.mean(preds == y_test))
    
    print('Finished iteration: ', i)

print('The accuracy is: ', sum(accuracy)/len(accuracy))
    

Finished iteration:  0
The accuracy is:  0.9881858299454463


# Apply the Model.

In [53]:
def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    return model.predict([diff])
    #return model.predict_proba([diff])

In [54]:
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2018
1,1102,Air Force,1985,2018
2,1103,Akron,1985,2018
3,1104,Alabama,1985,2018
4,1105,Alabama A&M,2000,2018


In [55]:
team1_name = 'North Carolina'
team2_name = 'Villanova'
team1_vector = get_season_data(teams[teams['TeamName'] == team1_name].values[0][0], 2016)
team2_vector = get_season_data(teams[teams['TeamName'] == team2_name].values[0][0], 2016)
#print('Probability that', team1_name,'wins:',predict_game(team1_vector, team2_vector, 0)[0])



# Predicting the NCAA Tournament

In [56]:
submission = pd.read_csv('./SampleSubmissionStage1.csv')
submission.head()

,ID,Pred
0,2014_1107_1110,0.5
1,2014_1107_1112,0.5
2,2014_1107_1113,0.5
3,2014_1107_1124,0.5
4,2014_1107_1140,0.5


In [57]:
def create_prediction():
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup_id = row['ID']
        year = matchup_id[0:4]
        team1_id = matchup_id[5:9]
        team2_id = matchup_id[10:14]
        team1_vector = get_season_data(int(team1_id), int(year))
        team2_vector = get_season_data(int(team2_id), int(year))
        pred = predict_game(team1_vector, team2_vector, 0)
        results[index][0] = matchup_id
        results[index][1] = pred[0]
        #results[index][1] = pred[0][1]
    results = pd.np.array(results)
    firstRow = [[0 for x in range(2)] for x in range(1)]
    firstRow[0][0] = 'ID'
    firstRow[0][1] = 'Pred'
    with open("result_1.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(firstRow)
        writer.writerows(results)
        

In [58]:
create_prediction()


KeyboardInterrupt: 

In [ ]:
model = GradientBoostingRegressor(n_estimators=100, max_depth=5)

In [ ]:
def dependency(predictions, test, stat, identifiers):
    difference = test[:, categories.index(stat)]
    plt.scatter(difference, predictions)
    plt.ylabel('Prob. that Team1 Wins')
    plt.xlabel(stat + 'Difference (Team1 - Team2)')
    plt.show()

In [ ]:
def feature_importance(identifiers):
    fi = pd.Series(model.feature_importances_, index = my_categories)
    fi /= fi.max()
    fi.sort()
    fi.plot(kind='barh')


In [ ]:
my_categories = ['Wins', 'PPG', 'PPGA', 'PowerConf', '3PG', 'APG', 'TOP', 'Conference Champ', \
             'Tourney Conference Champ', 'Seed', 'SOS', 'SRS', 'RPG', 'SPG', 'Tourney Appearances',\
                'National Championships', 'Location']

accuracy = []

for i in range(1):
    X_train, X_test, y_train, y_test = train_test_split(X_model, y_model)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    accuracy.append(np.mean(preds == y_test))
    
    print('Finished iteration: ', i)

print('The accuracy is: ', sum(accuracy)/len(accuracy))

In [ ]:
def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    return model.predict([diff])
    #return model.predict_proba([diff])

In [ ]:
def create_prediction():
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup_id = row['ID']
        year = matchup_id[0:4]
        team1_id = matchup_id[5:9]
        team2_id = matchup_id[10:14]
        team1_vector = get_season_data(int(team1_id), int(year))
        team2_vector = get_season_data(int(team2_id), int(year))
        pred = predict_game(team1_vector, team2_vector, 0)
        results[index][0] = matchup_id
        results[index][1] = pred[0]
        #results[index][1] = pred[0][1]
    results = pd.np.array(results)
    firstRow = [[0 for x in range(2)] for x in range(1)]
    firstRow[0][0] = 'ID'
    firstRow[0][1] = 'Pred'
    with open("result_2.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(firstRow)
        writer.writerows(results)
        

In [ ]:
create_prediction()

In [ ]:
model = GradientBoostingRegressor(n_estimators=100, max_depth=5)

In [ ]:
def dependency(predictions, test, stat, identifiers):
    difference = test[:, categories.index(stat)]
    plt.scatter(difference, predictions)
    plt.ylabel('Prob. that Team1 Wins')
    plt.xlabel(stat + 'Difference (Team1 - Team2)')
    plt.show()

In [ ]:
def feature_importance(identifiers):
    fi = pd.Series(model.feature_importances_, index = my_categories)
    fi /= fi.max()
    fi.sort()
    fi.plot(kind='barh')

In [ ]:
my_categories = ['Wins', 'PPG', 'PPGA', 'PowerConf', '3PG', 'APG', 'TOP', 'Conference Champ', \
             'Tourney Conference Champ', 'Seed', 'SOS', 'SRS', 'RPG', 'SPG', 'Tourney Appearances',\
                'National Championships', 'Location']

accuracy = []

for i in range(1):
    X_train, X_test, y_train, y_test = train_test_split(X_model, y_model)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    accuracy.append(np.mean(preds == y_test))
    
    print('Finished iteration: ', i)

print('The accuracy is: ', sum(accuracy)/len(accuracy))

In [ ]:
def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    return model.predict([diff])
    #return model.predict_proba([diff])

In [ ]:
def create_prediction():
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup_id = row['ID']
        year = matchup_id[0:4]
        team1_id = matchup_id[5:9]
        team2_id = matchup_id[10:14]
        team1_vector = get_season_data(int(team1_id), int(year))
        team2_vector = get_season_data(int(team2_id), int(year))
        pred = predict_game(team1_vector, team2_vector, 0)
        results[index][0] = matchup_id
        results[index][1] = pred[0]
        #results[index][1] = pred[0][1]
    results = pd.np.array(results)
    firstRow = [[0 for x in range(2)] for x in range(1)]
    firstRow[0][0] = 'ID'
    firstRow[0][1] = 'Pred'
    with open("result_3.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(firstRow)
        writer.writerows(results)

In [ ]:
create_prediction()

In [ ]:
model = GradientBoostingClassifier(n_estimators=100)

In [ ]:
def dependency(predictions, test, stat, identifiers):
    difference = test[:, categories.index(stat)]
    plt.scatter(difference, predictions)
    plt.ylabel('Prob. that Team1 Wins')
    plt.xlabel(stat + 'Difference (Team1 - Team2)')
    plt.show()

In [ ]:
def feature_importance(identifiers):
    fi = pd.Series(model.feature_importances_, index = my_categories)
    fi /= fi.max()
    fi.sort()
    fi.plot(kind='barh')

In [ ]:
my_categories = ['Wins', 'PPG', 'PPGA', 'PowerConf', '3PG', 'APG', 'TOP', 'Conference Champ', \
             'Tourney Conference Champ', 'Seed', 'SOS', 'SRS', 'RPG', 'SPG', 'Tourney Appearances',\
                'National Championships', 'Location']

accuracy = []

for i in range(1):
    X_train, X_test, y_train, y_test = train_test_split(X_model, y_model)
    results = model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    accuracy.append(np.mean(preds == y_test))
    
    print('Finished iteration: ', i)

print('The accuracy is: ', sum(accuracy)/len(accuracy))

In [ ]:
def predict_game(team_1_vector, team_2_vector, home):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    diff.append(home)
    return model.predict([diff])
    #return model.predict_proba([diff])

In [ ]:
def create_prediction():
    results = [[0 for x in range(2)] for x in range(len(submission.index))]
    for index, row in submission.iterrows():
        matchup_id = row['ID']
        year = matchup_id[0:4]
        team1_id = matchup_id[5:9]
        team2_id = matchup_id[10:14]
        team1_vector = get_season_data(int(team1_id), int(year))
        team2_vector = get_season_data(int(team2_id), int(year))
        pred = predict_game(team1_vector, team2_vector, 0)
        results[index][0] = matchup_id
        results[index][1] = pred[0]
        #results[index][1] = pred[0][1]
    results = pd.np.array(results)
    firstRow = [[0 for x in range(2)] for x in range(1)]
    firstRow[0][0] = 'ID'
    firstRow[0][1] = 'Pred'
    with open("result_4.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(firstRow)
        writer.writerows(results)

In [ ]:
create_prediction()

In [ ]:
def findBestK():
    k = (list)(i for i in range(1,200) if i%2 != 0)
    p = []
    for i in k:
        kmeans = KNeighborsClassifier(n_neighbors=k)
        kmeans.fit(X_train, y_train)
        results = kmeans.fit(X_train, y_train)
        preds = kmeans.predict(X_test)
        p.append(np.mean(preds == y_test))
    plt.plot(k, p)
    plt.xlabel('k')
    plt.ylabel('Accuracy')
    plt.title('Selecting k with the Elbow Method')
    plt.show()

    return p

In [ ]:
def neuralNetwork(loops):
    accuracy=[]
    dim = X_model.shape[1]
    for i in range(loops):
        X_train, X_test, y_train, y_test = train_test_split(X_model, y_model)

        model = Sequential()
        #model.add(Convolution1D(28, 3, border_mode='same', init='normal', input_shape=(dim, 1))) #28 1D filters of length 3
        model.add(Dense(128, input_dim=X_model.shape[1], activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(2, activation = 'softmax'))
        
        
#         model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))
#         model.add(Dense(4, activation='relu'))
#         model.add(Dense(1, activation='sigmoid'))

#         model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        #X_train = X_train.reshape((len(X_train), dim, 1))
        #X_test = X_test.reshape((len(X_test), dim, 1))
        y_train_categorical = np_utils.to_categorical(y_train)
        # TRAINING
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train_categorical, batch_size=64, nb_epoch=10,shuffle=True)
        preds = model.predict(X_test)
        results=[]
        for i in range(preds.shape[0]):
            if preds[i][1] < .5:
                results.append(0)
            else:
                results.append(1)
        accuracy.append(np.mean(results == Y_test))
    #accuracy.append(np.mean(predictions == Y_test))
    print "The accuracy is", sum(accuracy)/len(accuracy)